In [7]:
# for .env file for getting the key
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

##### Text Loader through Langchain

In [9]:
# importing TextLoader from documents_loaders of the langchain
from langchain.document_loaders import TextLoader

In [16]:
# creating the object of the TextLoader
loader = TextLoader("files/nvda_news.txt")
loader.load()

# checking the type of the loader
type(loader)

# checking the path of the loader
loader.file_path

'files/nvda_news.txt'

##### CSV Loader through Langchain

In [15]:
# importing CSVLoader from document_loaders.csv of the langchain
from langchain.document_loaders.csv_loader import CSVLoader